In [17]:
import numpy as np
import json_tricks


inputs = json_tricks.load('inputs.json')
answer = {}


# Task

You are given a linear hyperspace by 2 entities:
- a vector that belongs to the hyperspace $\mathbf x_0$
- a normal vector to this hyperspace $\mathbf w$

So that the hyperplane is given by equation:

$\left<\mathbf w, \mathbf x - \mathbf x_0\right> = 0$

You are also given another vector $\mathbf y$. 

Find distance from this vector to the given hyperplane.

In [18]:
def distance_to_hyperplane_1(y, w, x0):
    res = 0
    w = np.array(w)
    y = np.array(y)
    x0 = np.array(x0)
    numerator = np.abs(np.dot(w, y - x0))
    denominator = np.linalg.norm(w)
    res = numerator / denominator
    return res

In [19]:
answer['task1'] = []
for one_input in inputs['task1']:
    answer['task1'].append(
        distance_to_hyperplane_1(**one_input))

# Task

You are given a linear hyperspace by 2 entities:
- a vector that belongs to the hyperspace $\mathbf x_0$
- a set of direction vectors for this hyperspace $\mathbf a_1, \dots, \mathbf a_K$ in form of column-matrix 
    
    $A = \begin{bmatrix}
    | & | & & | \\
    \mathbf a_1 & \mathbf a_2 & \dots & \mathbf a_K \\
    | & | & & | \\
    \end{bmatrix}$

So that the hyperplane is given by equation:

$\mathbf x_0 + \sum_{k=1}^K \alpha_k \mathbf a_k = \mathbf 0$

You are also given another vector $\mathbf y$. 

Find distance from this vector to the given hyperplane.

In [20]:
def distance_to_hyperplane_2(y, A, x0):
    y = np.array(y, dtype=np.float64).flatten()
    x0 = np.array(x0, dtype=np.float64).flatten()
    A = np.array(A, dtype=np.float64)
    
    if A.size == 0:
        return np.linalg.norm(y - x0)
    
    # Normalize columns for better numerical stability
    norms = np.sqrt(np.sum(A * A, axis=0))
    norms[norms == 0] = 1
    A = A / norms
    
    # Calculate orthogonal basis for the column space of A
    Q = np.linalg.svd(A, full_matrices=False)[0]
    
    # Calculate the projection onto the orthogonal complement
    diff = y - x0
    proj = Q @ (Q.T @ diff)
    perp = diff - proj
    
    return np.linalg.norm(perp)

In [21]:
answer['task2'] = []
for one_input in inputs['task2']:
    answer['task2'].append(
        distance_to_hyperplane_2(**one_input))

# Task

You are given 2 linear hyperspaces, each defined by 2 entities:
- a vector that belongs to the hyperspace $\mathbf x_0$
- a set of direction vectors for this hyperspace $\mathbf a_1, \dots, \mathbf a_K$ in form of column-matrix 

So that each of the hyperplanes is given by equation:

$\mathbf x_0 + \sum_{k=1}^K \alpha_k \mathbf a_k = \mathbf 0$

Find distance between these hyperplanes.

In [22]:
def distance_to_hyperplane_3(A, x0, B, y0):
    # Convert inputs to float64 for better precision
    A = np.array(A, dtype=np.float64)
    x0 = np.array(x0, dtype=np.float64)
    B = np.array(B, dtype=np.float64)
    y0 = np.array(y0, dtype=np.float64)

    if A.size == 0 and B.size == 0:
        return np.linalg.norm(x0 - y0)

    # Normalize columns for better numerical stability
    if A.size > 0:
        A_norms = np.sqrt(np.sum(A * A, axis=0))
        A_norms[A_norms == 0] = 1
        A = A / A_norms

    if B.size > 0:
        B_norms = np.sqrt(np.sum(B * B, axis=0))
        B_norms[B_norms == 0] = 1
        B = B / B_norms

    # Combine matrices and solve least squares problem
    M = np.hstack([A, -B]) if (A.size > 0 and B.size > 0) else (A if A.size > 0 else -B)
    c = y0 - x0

    if M.size == 0:
        return np.linalg.norm(c)

    # Use SVD for better numerical stability
    U, s, Vh = np.linalg.svd(M, full_matrices=False)
    
    # Filter small singular values for numerical stability
    tol = 1e-10 * s[0]
    mask = s > tol
    U = U[:, mask]
    s = s[mask]
    Vh = Vh[mask]

    # Solve using SVD components
    z_hat = Vh.T @ (U.T @ c / s)
    proj = M @ z_hat
    perp = c - proj

    return np.linalg.norm(perp)

In [23]:
answer['task3'] = []
for one_input in inputs['task3']:
    answer['task3'].append(
        distance_to_hyperplane_3(**one_input))

In [24]:
json_tricks.dump(answer, '.answer.json')

'{"task1": [1.1611015532036637, 1.2169686348578783, 3.8171960993534175, 2.1577367083828145, 17.53449972356085, 13.749230523405476, 0.11441551070947108, 0.9750002110024923, 2.0481280123278074, 0.4178554470186725, 7.299732493572025, 5.405127802564358, 2.0541374149458744, 6.71079347281296, 3.03653640842388, 9.498185286556286, 3.7630890450319083, 1.8641906049463526, 10.670770020027398, 5.291772589866002, 0.912245460839306, 0.20732586478679277, 12.816310409006174, 1.6373225835853016, 6.291571550369559, 20.239709319388293, 16.466083091274903, 13.474719001142073, 5.2761044442380856, 0.7250110520819842, 4.519523979103389, 4.225904366704887, 4.775521858228876, 13.912436341952132, 7.24644943484495, 9.313787360469487, 1.0596535411727601, 5.682048493486052, 4.5807379490262905, 3.3300869947532497, 5.307227776030219, 5.6272294256742645, 4.913975701062781, 4.802205375719392, 6.501355432613329, 11.47165060247919, 1.1319709719592113, 4.803995448402746, 3.272807297357087, 15.777869833110813, 4.517699029